In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [14]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):
    try:
        price = soup.find("span", attrs={'id': 'priceblock_ourprice'}).string.strip()
    except AttributeError:
        try:
            price = soup.find("span", attrs={'id': 'priceblock_dealprice'}).string.strip()
        except AttributeError:
            try:
                # If price information is in a different span
                price = soup.find("span", class_='a-offscreen').string.strip()
            except AttributeError:
                price = ""
    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available



In [15]:
if __name__ == '__main__':
    # Add your user agent
    HEADERS = ({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36',
        'Accept-Language': 'en-US, en;q=0.5'
    })

    # The webpage URL
    URL = r"https://www.amazon.com/s?k=playstation+5&crid=3FGYTFQLYZAAP&sprefix=play%2Caps%2C203&ref=nb_sb_ss_w_hit-vc-lth_playstation-5_k0_1_4"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class': 'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
        links_list.append(link.get('href'))

    d = {"title": [], "price": [], "rating": [], "reviews": [], "availability": []}

    # Loop for extracting product details from each link
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'] = amazon_df['title'].replace('', np.nan)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [16]:
amazon_df.head(5)

,title,price,rating,reviews,availability
1,PlayStation 5 Digital Edition - CFI-1102B,Page 1 of 1,4.8 out of 5 stars,834 ratings,Not Available
2,EA SPORTS College Football 25 - PlayStation 5,$67.73,3.3 out of 5 stars,21 ratings,In Stock
3,PlayStation DualSense® Wireless Controller - M...,$69.00,4.7 out of 5 stars,"34,604 ratings",Only 11 left in stock - order soon.
4,WD_BLACK 2TB SN850P NVMe M.2 SSD Officially Li...,$199.99,4.8 out of 5 stars,"3,953 ratings",In Stock
5,$150 PlayStation Store Gift Card [Digital Code],$150.00,4.7 out of 5 stars,"250,198 ratings",Available now
